Import

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings("ignore")
%matplotlib inline

vote_df = pd.read_csv('../train.csv')
vote_df.head(3)

,index,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,43375,1.0,324,5.0,1067,5.0,359,1.0,709,1.0,...,1,1,1,1,1,0,1,0,1,1
1,4486,1.0,534,5.0,1555,5.0,2024,1.0,1097,1.0,...,1,1,1,1,1,0,1,0,1,1
2,10262,5.0,609,1.0,749,2.0,624,1.0,1833,3.0,...,1,1,0,1,1,0,1,0,1,0


In [4]:
vote_df = pd.read_csv('../train.csv')

#투표 여부 비율 확인
print(vote_df.voted.value_counts())

0    19918
1    16507
Name: voted, dtype: int64


In [5]:
#머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    vote_df.drop(['urban', 'gender', 'race', 'engnat', 'tp01','tp02', 'tp03','tp04','tp05','tp06','tp07','tp08','tp09','tp10',
            'wr_01','wr_02','wr_03','wr_04','wr_05','wr_06','wr_07','wr_08','wr_09',
            'wr_10','wr_11','wr_12','wr_13',
            'wf_01', 'wf_02', 'wf_03','hand'], axis = 1, inplace=True) 
    return df

In [6]:
def outlier_iqr(df=None, column= None, weight=1.5):
    data = df[df['voted']==1][column]
    q25, q75 = np.percentile(data.values, 25), np.percentile(data.values, 75)
    iqr = q75 - q25
    iqr_weight = iqr * weight
    lower = q25 - iqr_weight
    upper = q75 + iqr_weight
    outlier_index = data[(data < lower) | (data > upper)].index
    return outlier_index

In [7]:
#Q_E 함수
def replace_feature_Q(df):
    col_name = ['QaE', 'QbE', 'QcE', 'QdE','QeE','QfE','QgE','QhE','QiE','QjE','QkE','QlE','QmE','QnE','QoE','QpE','QqE','QrE','QsE','QtE']
    for col in col_name:
        outlier_index = outlier_iqr(vote_df, column = col)
        for i in outlier_index:
            vote_df[col].replace(vote_df.loc[i, col], vote_df[col].mean())
    
    scaler = MinMaxScaler()
    vote_df_QE = vote_df[col_name]
    scaler.fit(vote_df_QE)
    vote_df[col_name] = scaler.transform(vote_df_QE)
    return df

In [8]:
# age_group 함수
def replace_feature_age(df):
    vote_df['age_group'].replace('+70s', '60s',inplace=True)
    #age_group 레이블 인코딩 수행:
    feature = ['age_group']
    le = LabelEncoder()
    le = le.fit(df[feature])
    vote_df[feature] = le.transform(df[feature])
    return df

In [9]:
#familysize 함수
def fs_incodidng(df):
    def fs_labeling(df):
        label_list=[]
        for i in range(len(df)):
            if df.iloc[i]['familysize']==[2]:
                 label_list.append(7)
            elif df.iloc[i]['familysize']==[3]:
                 label_list.append(2)
            elif df.iloc[i]['familysize']==[1]:
                 label_list.append(5)
            elif df.iloc[i]['familysize']==[4]:
                 label_list.append(4)
            elif df.iloc[i]['familysize']==[5]:
                 label_list.append(3)
            elif df.iloc[i]['familysize']==[0]:
                 label_list.append(2)
            elif df.iloc[i]['familysize']==[6]:
                 label_list.append(1)
            else:
                label_list.append(0)
        return label_list
    vote_df['familysize'] = fs_labeling(vote_df)
    # 라벨링 후 0인 컬럼을 7로 변환(7이 가장 많은 비율을 차지하는 컬럼)
    vote_df['familysize'].replace(0,7,inplace=True)
    return df

In [10]:
# 0값 대체함수
def replace_feature(df):
    vote_df['education'].replace(0,2,inplace=True)
    vote_df['married'].replace(0,1,inplace=True)
    vote_df['familysize'].replace(0,7,inplace=True) 
    return df

In [11]:
# 원핫인코딩 함수
def OH_incording(df):
    pd.get_dummies(vote_df, columns=['education', 'married','religion'])
    return df

In [12]:
def transform_features(df):
    df = replace_feature_Q(df)
    df = replace_feature_age(df)
    df = drop_features(df)
    df = replace_feature(df)
    df = fs_incodidng(df)
    df = OH_incording(df)
    return df

In [13]:
# 원본데이터를 재로딩하고, 함수적용
vote_df = pd.read_csv('../train.csv')
y_vote_df = vote_df['voted']
X_vote_df = vote_df.drop('voted',axis=1)

X_vote_df = transform_features(X_vote_df)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_vote_df, y_vote_df, test_size=0.2, random_state = 11)